In [2]:
import numpy as np

In [ ]:
inputs = np.load("data/sudoku-extreme-1k-aug-1000-hint/train/all__inputs.npy")
print(inputs[0].reshape(9, 9)-1)
print(inputs[1].reshape(9, 9)-1)

[1 1 1 2 2 2 2 1 1 2 2 2 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 1 1 2 1 1 2 1
 2 1 2 2 1 1 2 2 1 2 2 2 1 2 1 1 1 2 2 1 2 2 2 2 1 2 1 2 2 2 1 2 1 1 1 2 2
 2 2 1 1 2 1 2 1 2 1 2 2 2 1 1 1 2 2 1 1 2 1 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2
 1 2 2 1 2 1 2 2 1 2 1 2 2 2 1 2 2 2 2 1 2 2 2 2 1 1 1 1 2 2 2 2 2 2 2 2 1
 2 1 2 1 2 2 1 2 1 1 2 1 2 1 2 1 1 1 1 1 1 4 2 2 2 2 1 1 1 2 2 2 1 2 2 2 2
 2 1 2 1 1 2 1 2 2 2 1 2 2 2 1 1 1 2 1 2 1 2 2 2 2 2 2 1 2 1 1 2 1 2 1 2 2
 1 2 2 2 2 2 2 1 1 1 1 2 1 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 1 2 1 1 1 2 2 2 2
 2 2 2 1 2 2 2 1 2 2 2 1 1 2 2 2 1 2 2 1 2 2 1 2 1 2 1 1 2 2 2 2 2 2 1 2 1
 1 2 2 2 2 1 2 2 2 1 1 1 2 2 1 1 2 2 1 2 1 2 2 1 1 1 2 2 2 2 1 2 1 2 2 2 2
 2 2 1 2 1 2 3 1 2 2 2 2 1 2 2 1 1 2 2 2 1 2 2 2 2 1 1 2 2 2 2 1 1 2 1 1 1
 2 1 1 2 2 1 2 1 1 2 2 1 2 2 2 2 2 1 2 2 2 2 2 1 2 1 1 2 2 1 2 1 1 2 2 2 1
 1 2 1 1 2 1 1 2 1 2 1 1 2 2 1 2 2 2 1 2 1 2 1 1 1 2 1 2 1 2 1 2 2 2 2 2 2
 1 1 2 2 2 2 2 2 1 2 1 2 2 1 1 1 2 1 1 1 2 1 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2
 1 2 2 2 2 1 2 1 1 2 2 2 